Dataframes Needed:
- Live games (Teams playing, scores, results)
- Box scores (Player stats)
- Lead tracker

Home Page
- Live game scores, Youtube videos of highlights

Game Page
- Box scores
- Matchup stats (lead tracker, team %s)
- Youtube highlight

Visualizations
- [Blank]

In [1]:
import nbapy
import datetime
import warnings
import pandas as pd
import dateutil.parser
from nbapy import game
from nbapy.scoreboard import Scoreboard

pd.set_option('max_columns', None)
pd.options.mode.chained_assignment = None

In [2]:
test_gameid = '0021901318'
test_date = '08-13-2020'

In [3]:
import constants

### Get Games Function

In [4]:
def get_games(datestring):
    """
    Input:
        datestring (str) - date of NBA games to grab
    
    Output:
        games (pandas df) - dataframe of NBA games, results, and teams playing
        
    """
    date = dateutil.parser.parse(datestring)
    scoreboard = Scoreboard(date.month, date.day, date.year)
    
    # Grab today's game information
    game_header = scoreboard.game_header()
    line_score = scoreboard.line_score()
    
    # Simplify columns
    games = game_header[['GAME_SEQUENCE', 'GAME_ID', 'GAME_STATUS_TEXT', 
                         'LIVE_PERIOD', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID',
                         'NATL_TV_BROADCASTER_ABBREVIATION']]
    score = line_score[['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'PTS']]
    
    # Dictionary of game values
    team_info = score.set_index('TEAM_ID')['TEAM_ABBREVIATION'].to_dict()
    team_pts = score.set_index('TEAM_ID')['PTS'].to_dict()
    
    # Map to games info
    games['HOME_TEAM_ABBREVIATION'] = games.HOME_TEAM_ID.map(team_info)
    games['VISITOR_TEAM_ABBREVIATION'] = games.VISITOR_TEAM_ID.map(team_info)
    games['HOME_TEAM_PTS'] = games.HOME_TEAM_ID.map(team_pts)
    games['VISITOR_TEAM_PTS'] = games.VISITOR_TEAM_ID.map(team_pts)
    
    # Create a winner column for web layout
    winners = []
    for i, value in games.iterrows():
        if (value["HOME_TEAM_PTS"] > value["VISITOR_TEAM_PTS"]):
            winners.append(value["HOME_TEAM_ABBREVIATION"])
        elif (value["HOME_TEAM_PTS"] < value["VISITOR_TEAM_PTS"]):
            winners.append(value["VISITOR_TEAM_ABBREVIATION"])
        else:
            winners.append(None)
    
    games['WINNER'] = winners
    
    return games

games = get_games(test_date)
games

,GAME_SEQUENCE,GAME_ID,GAME_STATUS_TEXT,LIVE_PERIOD,HOME_TEAM_ID,VISITOR_TEAM_ID,NATL_TV_BROADCASTER_ABBREVIATION,HOME_TEAM_ABBREVIATION,VISITOR_TEAM_ABBREVIATION,HOME_TEAM_PTS,VISITOR_TEAM_PTS,WINNER
0,1,0021901308,Final,4,1610612738,1610612764,None,BOS,WAS,90,96,WAS
1,2,0021901310,Final,4,1610612747,1610612758,None,LAL,SAC,122,136,SAC
2,3,0021901311,Final,4,1610612763,1610612749,None,MEM,MIL,119,106,MEM
3,4,0021901313,Final,4,1610612756,1610612742,TNT,PHX,DAL,128,102,PHX
4,5,0021901314,Final,4,1610612762,1610612759,TNT,UTA,SAS,118,112,UTA
5,6,0021901309,Final,4,1610612751,1610612757,TNT,BKN,POR,133,134,POR
6,7,0021901312,Final,4,1610612753,1610612740,None,ORL,NOP,133,127,ORL


In [11]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 12 columns):
GAME_SEQUENCE                       7 non-null int64
GAME_ID                             7 non-null object
GAME_STATUS_TEXT                    7 non-null object
LIVE_PERIOD                         7 non-null int64
HOME_TEAM_ID                        7 non-null int64
VISITOR_TEAM_ID                     7 non-null int64
NATL_TV_BROADCASTER_ABBREVIATION    3 non-null object
HOME_TEAM_ABBREVIATION              7 non-null object
VISITOR_TEAM_ABBREVIATION           7 non-null object
HOME_TEAM_PTS                       7 non-null int64
VISITOR_TEAM_PTS                    7 non-null int64
WINNER                              7 non-null object
dtypes: int64(6), object(6)
memory usage: 800.0+ bytes


### Get Live Games

Columns Needed
- GAME_STATUS_TEXT
- HOME_TEAM_ABBREVIATION
- VISITOR_TEAM_ABBREVIATION
- HOME_TEAM_PTS
- VISITOR_TEAM_PTS

In [13]:
def get_livegames():
    """
    Input:
        None
        
    Output:
        live_games (pandas df) - Dataframe showing results for today's NBA games
    """
    today = datetime.datetime.now()
    today_string = today.strftime("%m-%d-%Y")
    live_games = get_games(today_string)
    live_games = live_games.fillna(0)
    live_games[["HOME_TEAM_PTS", "VISITOR_TEAM_PTS"]] = live_games[["HOME_TEAM_PTS", "VISITOR_TEAM_PTS"]].astype(int)
    return live_games

live_games = get_livegames()
live_games

,GAME_SEQUENCE,GAME_ID,GAME_STATUS_TEXT,LIVE_PERIOD,HOME_TEAM_ID,VISITOR_TEAM_ID,NATL_TV_BROADCASTER_ABBREVIATION,HOME_TEAM_ABBREVIATION,VISITOR_TEAM_ABBREVIATION,HOME_TEAM_PTS,VISITOR_TEAM_PTS,WINNER
0,1,0041900113,1st Qtr,1,1610612751,1610612761,NBA TV,BKN,TOR,13,20,TOR
1,2,0041900163,4:00 pm ET,0,1610612762,1610612743,TNT,UTA,DEN,0,0,0
2,3,0041900123,6:30 pm ET,0,1610612755,1610612738,TNT,PHI,BOS,0,0,0
3,4,0041900153,9:00 pm ET,0,1610612742,1610612746,TNT,DAL,LAC,0,0,0


In [15]:
today = datetime.datetime.now()
today_string = today.strftime("%m/%d/%Y")
today_string

'08/21/2020'

### Get Box Scores

In [ ]:
def get_boxscore(game_id):
    """
    Input:
        game_id (str) - Game ID to pull box scores
    
    Output:
        box_score (pandas df) - Dataframe containing the box score results of Game ID
        
    """
    box_score = game.BoxScore(game_id).players_stats()
    return box_score

boxscore = get_boxscore(test_gameid)
boxscore.head()

### Get Team Stats

In [ ]:
def get_teamstats(game_id):
    """
    Input:
        game_id (str) - Game ID to pull team stats
        
    Output:
        team_stats (pandas df) - Dataframe showing the teams percentages
    
    """
    team_stats = game.BoxScore(game_id).team_stats()
    return team_stats

team_stats = get_teamstats(test_gameid)
team_stats

### Get Lead Tracker

In [ ]:
def get_linescore(game_id):
    """
    Input:
        game_id (str) - Game ID to pull line score
    
    Ouput:
        line_score (pandas df) - Dataframe showing the points by quarter
    
    """
    line_score = game.Info(game_id).line_score()
    return line_score

line_score = get_linescore(test_gameid)
line_score

### Get Standings

In [ ]:
scoreboard = Scoreboard()
east_standings = scoreboard.east_conf_standings_by_day()
west_standings = scoreboard.west_conf_standings_by_day()

In [ ]:
west_standings

In [ ]:
for i, value in east_standings.iterrows():
    print(value["TEAM"])

### Workspace

In [7]:
test_list = ['08/20/2020']
test_list

['08/20/2020']

In [8]:
test_list[0]

'08/20/2020'